In [ ]:
# Librerías
import random  # combinaciones aleatorias
import unicodedata  # normalizar acentos
import pygame #  sonido


# Inicializar pygame y el mezclador de sonido
pygame.init()
pygame.mixer.init()


# Cargar la canción 
pygame.mixer.music.load("terror.mp3") #tiene que estar en la misma carpeta que el juego
pygame.mixer.music.play(-1)  # Reproduce en bucle


# Palabras de terror escogidas
terror_words = [
    "espectro", "cripta", "ataúd", "tinieblas", "cementerio", "maldición",
    "fantasma", "exorcismo", "calavera", "posesión", "hechizo", "bruja",
    "ultratumba", "demonio", "sepulcro"
]


# Diccionario de frases de terror
terror_phrases = {
    "bienvenida": [
        "Bienvenida... al último juego de tu vida.",
        "La noche es oscura. Bienvenida, esta puede ser la última palabra que adivines.",
        "Has entrado. ¡Bienvenida! No podrás salir hasta que ganes."
    ],
    "acierto": [
        "Sí... pero ellos ya saben que estas aquí.",
        "Acertaste... pero algo se mueve en las sombras.",
        "Muy bien... por ahora sobrevives."
    ],
    "fallo": [
        "Has fallado... y la soga aprieta un poco más.",
        "No... y algo se arrastra hacia tí.",
        "Ese error te acerca al final."
    ],
    "victoria": [
        "Has sobrevivido... pero no por mucho tiempo.",
        "Ganaste... o eso crees.",
        "La palabra era tu salvación... ésta vez."
    ],
    "derrota": [
        "Has perdido... y ahora eres uno de ellos.",
        "No lograste escapar... la oscuridad te envuelve.",
        "Caiste... y nadie vendrá a ayudarte."
    ]
}


# Selección aleatoria de palabra
word = random.choice(terror_words)
word = unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8')
print(word) #PARA PROBAR QUE NUESTRO JUEGO FUNCIONA CORRECTAMENTE


# Mensajes iniciales
print(random.choice(terror_phrases["bienvenida"]))  #elección de palabras aleatorias con la librería random
print(f"Tu palabra contiene {len(word)} letras: {'_ ' * len(word)}") #son 6 intentos (7 - el primer dibujo inicial que aparece por defecto)


# Etapas del juego del ahorcado
hangman = [
    """
     ------
     |    |
     |
     |
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |    |
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|\\
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|\\
     |   /
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|\\
     |   / \\
    _|_
    """
]

# Variables de control del juego. Utilizamos conjuntos, más fáciles de utilizar a la hora de letras que se repiten 
correct_letters = set()
used_letters = set()
attempts_per_player = len(hangman) - 1  # intentos máximos por cada jugador. Variable para poder diferenciar los intentos de uno y de otro jugador
players = ["JUGADOR 1", "JUGADOR 2"]
score = {players[0]: 0, players[1]: 0} #marcador en forma de diccionario con los puntos de cada jugador
turn = 0  # empieza el Jugador 1
mistakes = {players[0]: 0, players[1]: 0}  # errores por jugador. Lo ponemos en forma de diccionario para que cuente los errores de ambos jugadores

print(f"Tienes {attempts_per_player} intentos para acertar la palabra. Vivir o morir. Tú decides.")

# Bucle principal del juego
while True:
    print("\n" + "-"*50)  # separador de turnos
    current_player = players[turn] # el jugador actual
    print(f"\nTURNO DE {current_player}")  #imprimimos para que se vea de quién es el turno
    print(hangman[mistakes[current_player]])  # dibujo según errores de este jugador

    # Mostrar palabra parcialmente revelada
    palabra_parcial = ""  #palabra parcial se refiere a la cadena formada por las letras que se van descubriendo y los guiones
    for letter_in_word in word: #recorremos con el iterable nuestra palabra a adivinar
        if letter_in_word.lower() in correct_letters:
            print(letter_in_word, end=" ")    #si se ha adivinado la letra, se imprime la letra con un espacio
            palabra_parcial += letter_in_word + " " #esto quiere decir que se añade es letra y el espacio a la cadena
        else:
            print("_", end=" ")   #si la letra no se adivina, lo que se imprime es el guión
            palabra_parcial += "_ " #esto quiere decir que en nuestra palabra parcial se añade ese guión
    print()

    #Prints para ver la palabra con guiones y letras descubiertas, para ver las letras que ya hemos usado y los intentos que le quedan a los jugadores
    print(f"Palabra actual: {palabra_parcial.strip()}")
    print(f"Letras usadas: {', '.join(sorted(used_letters)) if used_letters else 'Ninguna'}")
    print(f"A {current_player} le quedan {attempts_per_player - mistakes[current_player]} intentos") #intentos que quedan por jugador. Muestra los del Inicio de la ronda actual

 
    letter = input("Introduce una letra (o escribe 'salir' para terminar el juego): ").strip().lower()

    if letter == "salir":     # si en el input ponemos salir, hay un break y salimos del juego. Evitar darle manualmente a interrupt.
        print("Has salido del juego")
        print(f"La palabra era: {word}")
        break  #salimos del juego al escribir salir

    # Normalizar acento letra
    letter = unicodedata.normalize('NFKD', letter).encode('ASCII', 'ignore').decode('utf-8')

    # Validaciones
    if len(letter) != 1 or not letter.isalpha():   #no es válido un carácter que no es una letra o cuando escribimos más de una letra
        print(f"{letter} no es un caracter válido. Prueba de nuevo.")
        continue
    if letter in used_letters:
        print(f"Ya has usado la letra '{letter}'. Prueba con otra.")  #si escribimos letras que ya se han usado nos avisa
        continue

    used_letters.add(letter)

    # Comprobación letra en palabra
    if letter in word.lower(): #si está en la palabra, se añade al set de correct_leters y nos imprime que está en la palabra buscada
        correct_letters.add(letter)
        print(f"{random.choice(terror_phrases['acierto'])}. La '{letter}' está en la palabra buscada.")
       
    else:
        mistakes[current_player] += 1      #si letra no está palabra, se suma un fallo para el jugador que ha fallado, o sea el de ese turno
        print(f"{random.choice(terror_phrases['fallo'])}. La '{letter}' no está en la palabra buscada.") #frase aleatoria de fallo y de que la letra no está en la palabra
        failed_player = current_player  # Variable que guarda quién ha fallado. Esto sirve para luego decir que si el que ha fallado ya ha agotado todos sus intentos pues ha perdido el juego
        turn = 1 - turn  # cambia de turno sólo si hay fallo

        # Condición de derrota
        if mistakes[failed_player] == attempts_per_player: #si ya ha llegado el jugador al máximo de sus intentos permitidos ,se imprime mensaje de derrota
            print(f"{random.choice(terror_phrases['derrota'])}") #se imprimen frases aleatorias de derrota
            print(f"La palabra era: {word}")
            winner = players[1 - turn] # el jugador que no falló. Porque en cada turno, el jugador actual se corresponde con turn, el otro es 1-turn
            print(f"¡{winner} ES EL GANADOR DEL JUEGO DEL AHORCADO!")
            score[winner] += 1 #marcador que se suma al ganador
            print(f"El Marcador Final es: {players[0]}: {score[players[0]]} | {players[1]}: {score[players[1]]}") #nos hace un print de los puntos
            break

    # Condición de victoria
    if correct_letters >= set(word.lower()): #si el conjunto de letras correctas contiene al conjunto de letras en la palabra que tenemos que adivinar
        print(random.choice(terror_phrases["victoria"])) #se imprime frase aleatoria de victoria
        print(f"¡{current_player} ERES EL GANADOR DEL JUEGO DEL AHORCADO!") #se imprime que ese jugador de esa ronda ha ganado el juego
        print(f"La palabra era: {word}")
        score[current_player] += 1   #se suma al marcador del que ha ganado un punto
        print(f"Marcador actual: {players[0]}: {score[players[0]]} | {players[1]}: {score[players[1]]}")
        break


# Detener música y cerrar pygame
pygame.mixer.music.stop()
pygame.quit()